In [8]:
import os
import json
import pandas as pd
import traceback

In [39]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
key=os.getenv("OPENAI_API_KEY")

In [44]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)

In [45]:
llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001AEED9EEB10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001AEEDA009B0>, root_client=<openai.OpenAI object at 0x000001AEED78F8F0>, root_async_client=<openai.AsyncOpenAI object at 0x000001AEED9EEB70>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [56]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [92]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [93]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [94]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [95]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [96]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [97]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [98]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [99]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [100]:
file_path=r"D:\mcqgenerator\data.txt"

In [101]:
file_path

'D:\\mcqgenerator\\data.txt'

In [102]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [103]:
print(TEXT)

What is LangChain?
LangChain is a framework designed to build applications powered by Large Language Models (LLMs), such as OpenAI's GPT-3.5 or GPT-4. It provides tools, abstractions, and integrations that make it easier to create applications like chatbots, question-answering systems, summarization tools, and more.

LangChain's core philosophy is modularity and chainability, meaning it helps developers break down complex workflows into smaller, manageable, and reusable components.

Key Components of LangChain
LLMs (Large Language Models):

LangChain supports multiple LLMs (e.g., OpenAI, Hugging Face, Cohere).
It provides wrappers to interact with these models and configure parameters like temperature, model_name, and max_tokens.
Prompts:

LangChain offers tools to manage, format, and template prompts effectively.
You can define reusable prompt templates and dynamically inject variables.
Example:

python
Copy
Edit
from langchain.prompts import PromptTemplate

template = "Translate the 

In [104]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [116]:
NUMBER=5 
SUBJECT="LangChain"
TONE="hard"

In [117]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:What is LangChain?
LangChain is a framework designed to build applications powered by Large Language Models (LLMs), such as OpenAI's GPT-3.5 or GPT-4. It provides tools, abstractions, and integrations that make it easier to create applications like chatbots, question-answering systems, summarization tools, and more.

LangChain's core philosophy is modularity and chainability, meaning it helps developers break down complex workflows into smaller, manageable, and reusable components.

Key Components of LangChain
LLMs (Large Language Models):

LangChain supports multiple LLMs (e.g., OpenAI, Hugging Face, Cohere).
It provides wrappers to interact with these models and configure parameters like temperature, model_name, and max_tokens.
Prompts:

LangChain offers tools to manage, format, and template prompts effectively.
You can define reusable prompt templates and dynamically inject va

In [118]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3408
Prompt Tokens:2660
Completion Tokens:748
Total Cost:0.002452


In [119]:
quiz=response.get("quiz")
print(quiz)


{
    "1": {
        "mcq": "What is the core philosophy of LangChain?",
        "options": {
            "a": "Modularity and chainability",
            "b": "Flexibility and scalability",
            "c": "Efficiency and speed",
            "d": "Accuracy and precision"
        },
        "correct": "a"
    },
    "2": {
        "mcq": "Which component of LangChain is used to store the state of a conversation or process?",
        "options": {
            "a": "LLMs",
            "b": "Prompts",
            "c": "Memory",
            "d": "Chains"
        },
        "correct": "c"
    },
    "3": {
        "mcq": "What type of memory in LangChain is used for recent interactions?",
        "options": {
            "a": "Short-Term Memory",
            "b": "Long-Term Memory",
            "c": "Sequential Memory",
            "d": "Persistent Memory"
        },
        "correct": "a"
    },
    "4": {
        "mcq": "Which component of LangChain dynamically decides which action to take

In [121]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [122]:
quiz_table_data

[{'MCQ': 'What is the core philosophy of LangChain?',
  'Choices': 'a: Modularity and chainability | b: Flexibility and scalability | c: Efficiency and speed | d: Accuracy and precision',
  'Correct': 'a'},
 {'MCQ': 'Which component of LangChain is used to store the state of a conversation or process?',
  'Choices': 'a: LLMs | b: Prompts | c: Memory | d: Chains',
  'Correct': 'c'},
 {'MCQ': 'What type of memory in LangChain is used for recent interactions?',
  'Choices': 'a: Short-Term Memory | b: Long-Term Memory | c: Sequential Memory | d: Persistent Memory',
  'Correct': 'a'},
 {'MCQ': 'Which component of LangChain dynamically decides which action to take based on user input and available tools?',
  'Choices': 'a: Memory | b: Agents | c: Tools | d: Retrievers',
  'Correct': 'b'},
 {'MCQ': 'What is one of the common use cases of LangChain?',
  'Choices': 'a: Building cars | b: Cooking recipes | c: Chatbots | d: Weather forecasting',
  'Correct': 'c'}]

In [123]:
quiz=pd.DataFrame(quiz_table_data)

In [124]:
quiz.to_csv("machinelearning.csv",index=False)

In [125]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'01_22_2025_19_08_26'